In [ ]:
# Biblioteki

import pandas as pd 
import numpy as np
import pymysql
import mysql.connector
import datetime as dt
import os

print("Script is starting, the current time is", pd.to_datetime("today"))

# Variables

influ_window_days = 60
cc_window_days = 60
now = pd.to_datetime("today").
consumption_time = 45
excl = 60

# Import Vicidial

vicidial = pd.read_excel(r"C:\Users\stefa\Desktop\excele\vicidial\vicidial.xlsx")
vicidial

# Połączenie SQL

con = mysql.connector.connect(
  host="prod-replica.ceufrkrz6jt9.eu-central-1.rds.amazonaws.com",
  user="mdziekonski",
  password="ErhoUoD@46%EokESiR@1xq"
)

print(con)
mycursor = con.cursor()

# SQL QUERIES
# Clients Query

clients_query = ( """select so.customer_email, so.sales_channel, so.entity_id, so.created_at, 
                  MAX(so.created_at) OVER(PARTITION BY so.customer_email) AS 'customer_max',
                  MAX(so.created_at) OVER(PARTITION BY so.customer_email) - so.created_at as max_diff,
                  datediff(MAX(so.created_at) OVER(PARTITION BY so.customer_email), so.created_at) as days_diff,
                  base_subtotal, products
                  from magento.sales_order so
                  where so.state != 'canceled'
                  and so.status != 'canceled'
                  and so.is_valid = 1
                  and so.store_id = 1
                  and so.customer_email is not null
                  and so.customer_email not like '%healthlabs%'
                  and so.customer_email not like '%adequate%'
                  and so.coupon_code NOT IN ('Motywacja15', 'Dietoterapia10', '4US50', '4FR33', 
                  'PGFRIENDS35', '4THEM30', 'LORENC', 'LORENC10', 'LORENC25')""")
mycursor.execute(clients_query)
clients= pd.read_sql(clients_query, con)

# Stocks Query

stocks_query = ( """select so.entity_id, soi.product_id, soi.qty_ordered, 
                    case when products.consumption_time is null then 30 else products.consumption_time end as cons_time from sales_order so 
                    left join sales_order_item soi 
                    on so.entity_id = soi.order_id
                    left join
                    (
                    select soi.name, soi.product_id,
                    case when soi.product_id IN ('217', '248') then 180 when product_id IN ('174') then 120
                    when product_id IN ('66', '127', '227', '242', '206') then 60 
                    else 30 end as consumption_time from sales_order so 
                    left join sales_order_item soi 
                    on so.entity_id = soi.order_id 
                    where soi.product_type = 'simple'
                    and so.is_valid = 1
                    and so.store_id = 1
                    and so.created_at >= '2021-01-01'
                    group by 1,2) products
                    on soi.product_id = products.product_id
                    where so.is_valid = 1
                    and so.store_id = 1
                    and so.created_at >= '2021-01-01'""")
mycursor.execute(stocks_query)
stocks = pd.read_sql(stocks_query, con)

# Phone Query

phones_query = ("select email, phone from me_users mu ")
mycursor.execute(phones_query)
phones = pd.read_sql(phones_query, con)

# Influencers

clients['current_date'] = now 
clients['current_date'] = pd.to_datetime(clients['current_date'])
clients['influ_window_days'] = 60
clients['threshold'] = pd.to_datetime(clients.current_date) - pd.to_timedelta(pd.np.ceil(clients.influ_window_days), unit="D")
influ_exclusion = clients.drop_duplicates('customer_email')
influ_exclusion = clients[(clients['sales_channel'] == 'ecomm_influ') & (clients['threshold'] < clients['customer_max'])]
influ_exclusion['ex'] = 1
influ_exclusion[['ex', 'customer_email']]
influ_exclusion

# ❌ 60 ❌

vicidial_60 = vicidial
filter_list = ['CHUZNR', 'INFOSZ', 'MORE', 'AFTHRS',
'NA','ERI','DROP','DC','B','AFAX','ADC','AB','AA','A']
vicidial_60 = vicidial_60[vicidial_60.status.isin(filter_list)]
vicidial_60['window_days'] = 60
vicidial_60['date_60'] = pd.to_datetime(vicidial_60.call_date) - pd.to_timedelta(pd.np.ceil(vicidial_60.window_days), unit="D")
vicidial_60 = vicidial_60[(vicidial_60['call_date'] >= vicidial_60['date_60'])]
vicidial_exclusion_60 = vicidial_60.drop_duplicates('phone_number_dialed')
vicidial_exclusion_60 = (vicidial_60[['phone_number_dialed']])
vicidial_exclusion_60['phone_number_dialed'] = vicidial_exclusion_60['phone_number_dialed'].replace({"\\+48", ""})
vicidial_exclusion_60['phone_number_dialed'] = vicidial_exclusion_60['phone_number_dialed'].replace({"^0", ""})
vicidial_exclusion_60['phone_number_dialed'] = vicidial_exclusion_60['phone_number_dialed'].replace({" ", ""})
vicidial_exclusion_60['exx'] = 1
vicidial_exclusion_60

# 150

vicidial_150 = vicidial
filter_list = ['REZBEF','REZCEN','REZINP','REZMAP','REZNST','REZYGN']
vicidial_150 = vicidial_150[vicidial_150.status.isin(filter_list)]
vicidial_150['window_days'] = 150
vicidial_150['date_150'] = pd.to_datetime(vicidial_150.call_date) - pd.to_timedelta(pd.np.ceil(vicidial_150.window_days), unit="D")
vicidial_150 = vicidial_150[(vicidial_150['call_date'] >= vicidial_150['date_150'])]
vicidial_exclusion_150 = vicidial_150.drop_duplicates('phone_number_dialed')
vicidial_exclusion_150 = (vicidial_150[['phone_number_dialed']])
vicidial_exclusion_150['phone_number_dialed'] = vicidial_exclusion_150['phone_number_dialed'].replace({"\\+48", ""})
vicidial_exclusion_150['phone_number_dialed'] = vicidial_exclusion_150['phone_number_dialed'].replace({"^0", ""})
vicidial_exclusion_150['phone_number_dialed'] = vicidial_exclusion_150['phone_number_dialed'].replace({" ", ""})
vicidial_exclusion_150['exx'] = 1
vicidial_exclusion_150

# Wykluczenia 

nichko = vicidial
filter_list = ['DUBEL','NICHKO','ODMROZ','OSNIDO','POM',
#'PROPLA',
'ZASTAN'
#,'ODD'
]
nichko = nichko[nichko.status.isin(filter_list)]
nichko['window_days'] = 20000
nichko['date_20000'] = pd.to_datetime(nichko.call_date) - pd.to_timedelta(pd.np.ceil(nichko.window_days), unit="D")
nichko = nichko[(nichko['call_date'] >= nichko['date_20000'])]
vicidial_exclusion_nich = nichko.drop_duplicates('phone_number_dialed')
vicidial_exclusion_nich = (nichko[['phone_number_dialed']])
vicidial_exclusion_nich['phone_number_dialed'] = vicidial_exclusion_nich['phone_number_dialed'].replace({"\\+48", ""})
vicidial_exclusion_nich['phone_number_dialed'] = vicidial_exclusion_nich['phone_number_dialed'].replace({"^0", ""})
vicidial_exclusion_nich['phone_number_dialed'] = vicidial_exclusion_nich['phone_number_dialed'].replace({" ", ""})
vicidial_exclusion_nich['exx'] = 1
vicidial_exclusion_nich

# Połączenie Wykluczeń

vicidial_exclusion = pd.concat([vicidial_exclusion_nich, vicidial_exclusion_60, vicidial_exclusion_150])
vicidial_exclusion = vicidial_exclusion.drop_duplicates()
vicidial_exclusion

# Order Lenght

stocks['product_length'] = stocks['cons_time'] * stocks['qty_ordered']
order_delay = stocks.group_by(['entity_id']).agg('product_length': 'max',  
                                                           'product_id': 'count', avgg = mean(product_length))

df %>% groupby(col1) %>% summarize(col2_agg=max(col2), col3_agg=min(col3))

df.groupby('col1').agg({'col2': 'max', 'col3': 'min'})


clients = clients.merge(order_delay, on='entity_id', how='left')
clients['avgg'][is.na(clients$avgg)] <- 30
clients$chec1 <- ifelse(as.Date(clients$customer_max) + clients$avgg < as.Date(now), 0, 1)
clients <- clients %>% filter(max_diff == 0) 

# Dataset Creation


df_main = clients.group_by(['customer_email']).agg(max_date = as.Date(max(customer_max))
            , exclusions = sum(chec1)
            ) %>% arrange(desc(max_date)) %>% 
  filter(exclusions == 0) 

phones['phone'] = phones['phone'].replace({"\\+48", ""})
phones['phone'] = phones['phone'].replace({"\\+", ""})
phones['phone'] = phones['phone'].replace({"^0", ""})
phones['phone'] = phones['phone'].replace({"\\?", ""})
phones['phone'] = phones['phone'].replace({" ", ""})
phones = phones.dropna(how='all')
phones.phone = pd.to_numeric(phones.phone, errors='coerce').fillna(0).astype(np.int64)
phones = phones[phones['phone'].between(400000000, 960000000)]
df_main = df_main.merge(phones, left_on='email', right_on='customer_email', how='left')
df_main = df_main.merge(vicidial_exclusion, left_on='phone', right_on='phone_number_dialed', how='left')
df_main = df_main.merge(influ_exclusion, left_on='customer_email', right_on='customer_email', how='left')

os.chdir("C:\Users\stefa\Desktop\csvki\model_cc")
files <- list.files(pattern = '\\.csv$', full.names = TRUE)

all_data <- do.call(rbind, lapply(files, function(x) 
  transform(read.csv(x), File = basename(x))))

all_data <- all_data[2]
all_data <- all_data %>% distinct()
all_data$z <- 1

all_data$phone <- as.character(all_data$phone)
df_main <- df_main %>% left_join(all_data, by = c("phone")) 

df_final <- df_main %>% 
 filter(is.na(ex)) %>% 
  filter(is.na(exx)) %>% 
  filter(is.na(z))%>%
  arrange(desc(max_date)) 